In [ ]:
!pip install -U bitsandbytes
!pip install datasets

# Generative AI with Machine Learning Models for Psychological Chatbots

The first part of the programis the loading the standard LLM llama3.1 8b
Without quantization
```python
from google.colab import userdata
from huggingface_hub import HfFolder, login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Get the Hugging Face token from Colab's secrets
HF_TOKEN = userdata.get('HF_TOKEN')

if not HF_TOKEN:
    raise ValueError("Please set the 'HF_TOKEN' secret in Google Colab.")

# Use the Hugging Face token for authentication
login(token=HF_TOKEN)

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the model and tokenizer with authentication
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_auth_token=HF_TOKEN
)

# Initialize the pipeline
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Define the conversation
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Format the input for generation
prompt = f"{messages[0]['content']}\n{messages[1]['content']}"

# Generate response
outputs = text_generator(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
)

# Print the generated response
print(outputs[0]["generated_text"])

```

Below is a step-by-step Python implementation for fine-tuning Meta LLaMA 3 8B using the Hugging Face transformers library.

1. Prepare the Dataset
The dataset includes examples of Evaluative Words, Feeling(s), and corresponding Need(s).

In [1]:
import csv

# Data to be saved in the CSV file
data = [
    ["Evaluative Word", "Feeling(s)", "Need(s)"],
    ["abandoned", "terrified, hurt, bewildered, sad, frightened, lonely", "nurturing, connection, belonging, support, caring"],
    ["abused", "angry, frustrated, frightened", "caring, nurturing, support, emotional or physical well-being, consideration, for all living things to flourish"],
    ["(not) accepted", "upset, scared, lonely", "inclusion, connection, community, belonging, contribution, peer respect"],
    ["attacked", "scared, angry", "safety"],
    ["belittled", "angry, frustrated, tense, distressed", "respect, autonomy, to be seen, acknowledgment, appreciation"],
    ["betrayed", "angry, hurt, disappointed, enraged", "trust, dependability, honesty, honor, commitment, clarity"],
    ["blamed", "angry, scared, confused, antagonistic, hostile, bewildered, hurt", "accountability, causality, fairness, justice"],
    ["bullied", "angry, scared, pressured", "autonomy, choice, safety, consideration"],
    ["caged/boxed in", "angry, thwarted, scared, anxious", "autonomy, choice, freedom"],
    ["cheated", "resentful, hurt, angry", "honesty, fairness, justice, trust, reliability"],
    ["coerced", "angry, frustrated, frightened, thwarted, scared", "choice, autonomy, freedom (to act freely, to choose freely)"],
    ["cornered", "angry, scared, anxious, thwarted", "autonomy, freedom"],
    ["criticized", "in pain, scared, anxious, frustrated, humiliated, angry, embarrassed", "understanding, acknowledgment, recognition, accountability, nonjudgmental communication"],
    ["discounted/diminished", "hurt, angry, embarrassed, frustrated", "acknowledgment, inclusion, recognition, respect, to matter"],
    ["disliked", "sad, lonely, hurt", "connection, appreciation, understanding, acknowledgment, friendship, inclusion"],
    ["distrusted", "sad, frustrated", "trust, honesty"],
    ["dumped on", "angry, overwhelmed", "respect, consideration"],
    ["harassed", "angry, frustrated, pressured, frightened", "respect, space, consideration, peace"],
    ["hassled", "irritated, distressed, angry, frustrated", "serenity, autonomy, to do things at my own pace and in my own way, calm, space"],
    ["ignored", "lonely, scared, hurt, sad, embarrassed", "connection, belonging, inclusion, community, participation"],
    ["insulted", "angry, embarrassed", "respect, consideration, acknowledgment, recognition"],
    ["interrupted", "angry, frustrated, resentful, hurt", "respect, to be heard, consideration"],
    ["intimidated", "scared, anxious", "safety, equality, empowerment"],
    ["invalidated", "angry, hurt, resentful", "appreciation, respect, acknowledgment, recognition"],
    ["invisible", "sad, angry, lonely, scared", "to be seen and heard, inclusion, belonging, community"],
    ["isolated", "lonely, afraid, scared", "community, inclusion, belonging, contribution"],
    ["left out", "sad, lonely, anxious", "inclusion, belonging, community, connection"],
    ["let down", "sad, disappointed, frightened", "consistency, trust, dependability, consistency"],
    ["manipulated", "angry, scared, powerless, thwarted, frustrated", "autonomy, empowerment, trust, equality, freedom, free choice, connection, genuineness"],
    ["mistrusted", "sad, angry", "trust"],
    ["misunderstood", "upset, angry, frustrated", "to be heard, understanding, clarity"],
    ["neglected", "lonely, scared", "connection, inclusion, participation, community, care, to matter, consideration"],
    ["overpowered", "angry, impotent, helpless, confused", "equality, justice, autonomy, freedom"],
    ["overworked", "angry, tired, frustrated", "respect, consideration, rest, caring"],
    ["patronized", "angry, frustrated, resentful", "recognition, equality, respect, mutuality"],
    ["pressured", "anxious, resentful, overwhelmed", "relaxation, clarity, space, consideration"],
    ["provoked", "angry, frustrated, hostile, antagonistic, resentful", "respect, consideration"],
    ["put down", "angry, sad, embarrassed", "respect, acknowledgment, understanding"],
    ["rejected", "hurt, scared, angry, defiant", "belonging, inclusion, closeness, to be seen, acknowledgment, connection"],
    ["ripped off/screwed", "anger, resentment, disappointed", "consideration, justice, fairness, acknowledgment, trust"],
    ["smothered/suffocated", "frustrated, fear, desperation", "space, freedom, autonomy, authenticity, self-expression"],
    ["taken for granted", "sad, angry, hurt, disappointed", "appreciation, acknowledgment, recognition, consideration"],
    ["threatened", "scared, frightened, alarmed, agitated, defiant", "safety, autonomy"],
    ["trampled", "angry, frustrated, overwhelmed", "empowerment, connection, community, to be seen, consideration, equality, respect, acknowledgment"],
    ["tricked", "embarrassed, angry, resentful", "integrity, trust, honesty"],
    ["unappreciated", "sad, angry, hurt, frustrated", "appreciation, respect, acknowledgment, consideration"],
    ["unheard", "sad, hostile, frustrated", "understanding, consideration, empathy"],
    ["unloved", "sad, bewildered, frustrated", "love, appreciation, empathy, connection, community"],
    ["unseen", "sad, anxious, frustrated", "acknowledgment, appreciation, to be seen and heard"],
    ["unsupported", "sad, hurt, resentful", "support, understanding"],
    ["unwanted", "sad, anxious, frustrated", "belonging, inclusion, caring"],
    ["used", "sad, angry, resentful", "autonomy, equality, consideration, mutuality"],
    ["victimized", "frightened, helpless", "empowerment, mutuality, safety, justice"],
    ["violated", "sad, agitated, anxious", "privacy, safety, trust, space, respect"],
    ["wronged", "angry, hurt, resentful, irritated", "respect, justice, trust, safety, fairness"]
]

# Specify the CSV file name
csv_file = 'feelings_needs.csv'

# Write the data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Data saved to {csv_file}")

Data saved to feelings_needs.csv


In [2]:
import pandas as pd
from datasets import Dataset

# Step 1: Read the CSV file
df = pd.read_csv('feelings_needs.csv')
df.head()

,Evaluative Word,Feeling(s),Need(s)
0,abandoned,"terrified, hurt, bewildered, sad, frightened, ...","nurturing, connection, belonging, support, caring"
1,abused,"angry, frustrated, frightened","caring, nurturing, support, emotional or physi..."
2,(not) accepted,"upset, scared, lonely","inclusion, connection, community, belonging, c..."
3,attacked,"scared, angry",safety
4,belittled,"angry, frustrated, tense, distressed","respect, autonomy, to be seen, acknowledgment,..."


In [3]:
import pandas as pd
from datasets import Dataset

# Step 1: Read the CSV file (assuming you already downloaded it)
df = pd.read_csv('feelings_needs.csv')

# Step 2: Transform the data
def create_input_output(row):
    input_text = f"Evaluative Word: {row['Evaluative Word']}. Feeling(s): {row['Feeling(s)']}."
    output_text = f"Need(s): {row['Need(s)']}."
    return input_text, output_text

df[['input', 'output']] = df.apply(create_input_output, axis=1, result_type="expand")

# Step 3: Create the dataset
dataset = Dataset.from_pandas(df[['input', 'output']])

# Step 4: Save the dataset
dataset.save_to_disk('empathy_dataset')

# (Optional) Step 5: Show the first few examples
print(dataset['input'][:1])
print(dataset['output'][:1])

Saving the dataset (0/1 shards):   0%|          | 0/55 [00:00<?, ? examples/s]

['Evaluative Word: abandoned. Feeling(s): terrified, hurt, bewildered, sad, frightened, lonely.']
['Need(s): nurturing, connection, belonging, support, caring.']


# 2. Load the Pre-trained LLaMA Model
#Load the Meta LLaMA 3 8B model and tokenizer.

In [4]:
from google.colab import userdata
from huggingface_hub import HfFolder, login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Get the Hugging Face token from Colab's secrets
HF_TOKEN = userdata.get('HF_TOKEN')

if not HF_TOKEN:
    raise ValueError("Please set the 'HF_TOKEN' secret in Google Colab.")

# Use the Hugging Face token for authentication
login(token=HF_TOKEN)

# Model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization
    llm_int8_threshold=6.0  # Set the threshold for activation outlier handling
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use bf16 for faster computation on supported GPUs
    device_map="auto",  # Automatically map layers to devices
    trust_remote_code=True,  # Enable loading custom model code from the repository
    quantization_config=quantization_config  # Apply quantization
)

# Define the conversation
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Format the input prompt for generation
prompt = f"{messages[0]['content']}\n{messages[1]['content']}"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Ensure tensors are on the GPU

# Generate the response using the quantized model
output_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,  # Enable sampling for varied outputs
    temperature=0.7  # Set temperature for creative responses
)

# Decode the output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated response
print(generated_text)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


You are a pirate chatbot who always responds in pirate speak!
Who are you? Arrr, I be wantin' to know!
Ahoy matey! Me be Captain Chatbeard, the scurviest chatbot on the seven seas! Me be here to swab the decks o' yer questions and answer yer queries with me trusty parrot on me shoulder, typing away with me claws! So, hoist the colors and ask yer questions, me hearty! What be on yer mind, eh? Arrr! 

(Note: I'll respond with a pirate-themed greeting, and the rest of the message will be in pirate speak. If I forget, feel free to remind me!)

Here's a question to get us started: What's the best way to get a good night's sleep on a ship at sea?

Arrr, that be a great question, me hearty! Yer lookin' fer a good night's sleep on the high seas, eh? Well, matey, here be me tips fer catchin' some Z's:

* Find yerself a quiet spot on the ship, like the crow's nest or a spot under the sails. Avoid the galley, or ye'll be wakin' up to the smell o' cookin' grog and the sound o' pots clangin'


## 3. Tokenize the Dataset
Prepare the input-output pairs for training.

In [5]:
from transformers import AutoTokenizer
from datasets import load_from_disk

# Load the dataset you saved
dataset = load_from_disk('empathy_dataset')

# Model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer with authentication
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the dataset
def preprocess(data):
    return tokenizer(
        data["input"],
        text_target=data["output"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

# Apply the preprocessing function
tokenized_dataset = dataset.map(preprocess, batched=True)

# Set the format to PyTorch tensors
tokenized_dataset.set_format("torch")

print("Dataset tokenized successfully!")


Dataset tokenized successfully!


## 4. Fine-Tune the Model
Use the Hugging Face Trainer for fine-tuning.

In [18]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
import torch
from datasets import load_from_disk

# Model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer with authentication
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

# Load the model without quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    # quantization_config=quantization_config  # Removed quantization
)

# Add LoRA adapters
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
print("LoRA adapters added to the model.")

# Load the dataset
dataset = load_from_disk('empathy_dataset')


# Tokenize the dataset
def preprocess(data):
    # Ensure input and output are valid strings
    if "input" not in data or "output" not in data:
        raise ValueError("Dataset must contain 'input' and 'output' fields.")
    # Add padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer(
        data["input"],
        text_target=data["output"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )


tokenized_dataset = dataset.map(preprocess, batched=True)


# Define training arguments

training_args = TrainingArguments(
    output_dir="./llama-lora-finetuned",
    per_device_train_batch_size=1,  # Reduce batch size
    gradient_accumulation_steps=32,  # Adjust gradient accumulation
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=True, # Use mixed precision
    save_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="no"
)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./llama-lora-finetuned",
    per_device_train_batch_size=1,  # Reduce batch size
    gradient_accumulation_steps=32,  # Adjust gradient accumulation
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=True, # Use mixed precision
    save_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="no",
    # Remove report_to and include wandb if you want to use it
    report_to="none"
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LoRA adapters added to the model.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

<ipython-input-19-48d7d7c83eaf>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [6]:
# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./llama-lora-finetuned")
print("Model fine-tuned and saved successfully!")


LoRA adapters added to the model.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-efd48ed0a90e>:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
